# Mini-DES: A simpler version of DES
## Implementation in python

In [121]:
# imports
import numpy as np

In [122]:
# define the keys (e1, e2) and the s-boxes (s1, s2) here
e1 = np.array([1,0,0,1,1,0])
e2 = np.array([0,0,1,1,1,0])
s1 = np.array([[0,3,1,2],[2,3,0,1]])
s2 = np.array([[2,1,3,0],[0,1,2,3]])

In [123]:
# get the text from the user; output: an array of integers marking each character of the message
def getMessageAsInts():
  message = input("Enter your message: ")
  message = bytes(message, 'utf-8')
  message = np.array(bytearray(message))
  return message

In [124]:
# takes 4 bits and expands them to a 6-bit long sequence
def expand(bits):
  expanded = np.array([])
  expanded = np.append(expanded, bits[0])
  expanded = np.append(expanded, bits[3])
  expanded = np.append(expanded, bits[2])
  expanded = np.append(expanded, bits[1])
  expanded = np.append(expanded, bits[3])
  expanded = np.append(expanded, bits[0])
  return expanded

In [125]:
# XOR gate for bit sequences; output: a bitstring with the length of bits1 and bits2
def XOR(bits1, bits2):
  if(len(bits1)!=len(bits2)):
    print("ERROR: XOR: sequences are not the same length.")
    exit(1)
  else:
    result = np.array([])
    for x in range(len(bits1)):
      result = np.append(result, int(bits1[x]) ^ int(bits2[x]))
    return result

In [126]:
# SBoxes() takes 6 bits and processes them with the help of the s-boxes; output: 4 bits
def SBoxes(bits):
  firstHalf = bits[:3]
  secondHalf = bits[3:]
  result = ""

  result1 = bin(s1[int(firstHalf[0])][int(firstHalf[1]*2+firstHalf[2])])[2:].zfill(2)
  result2 = bin(s2[int(secondHalf[0])][int(secondHalf[1]*2+secondHalf[2])])[2:].zfill(2)

  result = result1 + result2

  return result

In [127]:
# verbose encryption routine
def encryptBits(bits):
  left = bits[:4]
  right = bits[4:]
  temp = np.array([]) # temp holds the temporary result
  bits_encrypted = "" # bits_encrypted holds the final result

  # Mini-DES algorithm, first round
  temp = expand(right)
  temp = XOR(temp, e1)
  temp = SBoxes(temp)
  temp = XOR(left, temp)
  left = right
  right = temp
  # second round
  temp = expand(right)
  temp = XOR(temp, e2)
  temp = SBoxes(temp)
  temp = XOR(left, temp)
  left = right
  right = temp

  # merge left and right half into a string (8-bit long)
  for b in left:
    bits_encrypted += str(int(b))
  for b in right:
    bits_encrypted += str(int(b))

  return bits_encrypted

In [128]:
# verbose decryption routine
def decryptBits(bits):
  left = bits[:4]
  right = bits [4:]
  temp = np.array([]) # temp holds the temporary result
  bits_decrypted = "" # bits_decrypted holds the final result

  # Mini-DES, now with reversed order of the keys
  temp = expand(left)
  temp = XOR(temp, e2)
  temp = SBoxes(temp)
  temp = XOR(temp, right)
  right = left
  left = temp

  temp = expand(left)
  temp = XOR(temp, e1)
  temp = SBoxes(temp)
  temp = XOR(temp, right)
  right = left
  left = temp

  # merge left and right half into a string (8-bit long)
  for b in left:
    bits_decrypted += str(int(b))
  for b in right:
    bits_decrypted += str(int(b))

  return bits_decrypted

### Now let's encrypt a message:

In [129]:
msg_integers = getMessageAsInts()
msg_encrypted = ""
msg_encoded = ""

for integer in msg_integers:
  bits = bin(integer)       # convert integer to binary string
  bits = bits[2:].zfill(8)  # remove '0b' and pad the string with zeros on the left; bits is now 8-bit long
  msg_encoded += bits
  bits_encrypted = encryptBits(bits)  # bits_encrypted is a 8-bit long binary string as well
  msg_encrypted += bits_encrypted

print("Message in numbers       :", msg_integers)
print("Message in bits          :", msg_encoded)
print("Encrypted message in bits:", msg_encrypted)

Message in numbers       : [ 84 104 105 115  32 105 115  32  97  32 116 101 115 116  33]
Message in bits          : 010101000110100001101001011100110010000001101001011100110010000001100001001000000111010001100101011100110111010000100001
Encrypted message in bits: 111011000101001000111101001101111000111100111101001101111000111101100010100011111100100001001010001101111100100000100011


### Decrypt the encrypted message:

In [130]:
msg_decrypted = ""

for i in range(0, len(msg_encrypted), 8):
  substring = msg_encrypted[i:i+8]
  substring_decrypted = decryptBits(substring)
  substring_decrypted = chr(int(substring_decrypted,2)) # convert 8-bit long bitstring to char (utf-8)
  msg_decrypted += substring_decrypted

print("Decrypted message:", msg_decrypted)

Decrypted message: This is a test!


### What you get when you decode the encrypted message without decrypting it:

In [131]:
msg_encrypted_decoded = ""

for i in range(0, len(msg_encrypted), 8):
  substring = msg_encrypted[i:i+8]
  substring_decoded = chr(int(substring,2)) # convert 8-bit long bitstring to char (utf-8)
  msg_encrypted_decoded += substring_decoded

print("Encrypted message decoded:", msg_encrypted_decoded)

Encrypted message decoded: ìR=7=7bÈJ7È#
